# Preprocessing and Baseline Modeling

This notebook implements a reproducible preprocessing pipeline and establishes a baseline model
using insights derived from the EDA phase.

## Data Loading

In [1]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/raw")

train_df = pd.read_csv(DATA_DIR / "train.csv")
test_df = pd.read_csv(DATA_DIR / "test.csv")

train_df.shape, test_df.shape

((1460, 81), (1459, 80))

## Problem Setup

- Task: Supervised regression
- Target variable: SalePrice
- Evaluation metric: RMSE
- Validation strategy: Hold-out split (80/20)

In [2]:
y = train_df["SalePrice"]
X = train_df.drop(columns=["SalePrice", "Id"])

## Train–Validation Split

We split the training data into train and validation sets to evaluate models before final training.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1168, 79), (292, 79), (1168,), (292,))

## Feature Groups

Separate numerical and categorical features for preprocessing.

In [4]:
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

len(numerical_features), len(categorical_features)

(36, 43)

## Numerical Feature Preprocessing

- Median imputation for missing values
- Log transformation for skewed features
- Standard scaling

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
import numpy as np

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

## Categorical Feature Preprocessing

- Constant imputation for missing values
- Ordinal encoding for categorical features

In [6]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="None")),
    ("encoder", OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=True
    ))
])

## Full Preprocessing Pipeline

Combine numerical and categorical preprocessing using ColumnTransformer.

In [7]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## Baseline Model

Train a Ridge Regression model using the full preprocessing pipeline.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [9]:
baseline_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0))
])

In [10]:
baseline_model.fit(X_train, y_train)
y_val_pred = baseline_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
rmse

np.float64(29738.614259190308)

## Baseline Model with Log-Transformed Target

Train the same model using a log-transformed target to handle skewness.

In [11]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

baseline_model.fit(X_train, y_train_log)

y_val_pred_log = baseline_model.predict(X_val)
y_val_pred = np.expm1(y_val_pred_log)

rmse_log = np.sqrt(mean_squared_error(y_val, y_val_pred))
rmse_log

np.float64(23798.854338040815)

## Baseline Summary

- Ridge regression with raw target resulted in RMSE ≈ 37.9k
- Log-transforming the target reduced RMSE to ≈ 29.4k
- This confirms strong skewness in SalePrice
- All future models will be trained on log1p(SalePrice)

## Cross-Validation Evaluation

Evaluate the baseline model using 5-fold cross-validation for a more robust performance estimate.

In [12]:
from sklearn.model_selection import KFold, cross_val_score
cv_scores = cross_val_score(
    baseline_model,
    X,
    np.log1p(y),
    scoring="neg_root_mean_squared_error",
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1
)

cv_rmse_log_mean = -cv_scores.mean()
cv_rmse_log_std = cv_scores.std()

cv_rmse_log_mean, cv_rmse_log_std

(np.float64(0.14814980518170512), np.float64(0.04083016562658464))

## Cross-Validation (Log Space)

- 5-fold CV log-RMSE ≈ 0.148 ± 0.041
- Evaluation performed in log space due to target skewness

## ElasticNet Model (Cross-Validated)

In [13]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {
    "regressor__alpha": [0.001, 0.01, 0.1, 1.0],
    "regressor__l1_ratio": [0.1, 0.5, 0.9]
}

In [15]:
elasticnet_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", ElasticNet(
        max_iter=5000,
        random_state=42
    ))
])

In [16]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=elasticnet_pipeline,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1
)

grid_search.fit(X, np.log1p(y))

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'regressor__alpha': [0.001, 0.01, ...], 'regressor__l1_ratio': [0.1, 0.5, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_root_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",KFold(n_split... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold an

In [17]:
best_score = -grid_search.best_score_
best_params = grid_search.best_params_

best_score, best_params

(np.float64(0.14368587413834397),
 {'regressor__alpha': 0.001, 'regressor__l1_ratio': 0.5})

## Model Comparison

- Ridge CV log-RMSE ≈ 0.148
- ElasticNet CV log-RMSE ≈ 0.144

ElasticNet provides a modest but consistent improvement by combining L1 and L2 regularization,
helping with multicollinearity and feature selection in one-hot encoded features.

In [18]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

In [19]:
numeric_tree = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

In [20]:
categorical_tree = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="None")),
    ("encoder", OrdinalEncoder(
        handle_unknown="use_encoded_value",
        unknown_value=-1
    ))
])

In [21]:
preprocessor_tree = ColumnTransformer(
    transformers=[
        ("num", numeric_tree, numerical_features),
        ("cat", categorical_tree, categorical_features)
    ]
)

In [22]:
hgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor_tree),
    ("regressor", HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=6,
        max_iter=300,
        random_state=42
    ))
])

In [23]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    hgb_pipeline,
    X,
    np.log1p(y),
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1
)

hgb_log_rmse_mean = -cv_scores.mean()
hgb_log_rmse_std = cv_scores.std()

hgb_log_rmse_mean, hgb_log_rmse_std

(np.float64(0.13380432429040437), np.float64(0.018917693721084365))

## Model Comparison

- Ridge (CV log-RMSE): ≈ 0.148
- ElasticNet (CV log-RMSE): ≈ 0.144
- HistGradientBoosting (CV log-RMSE): ≈ 0.134

Tree-based models outperform linear models by capturing non-linear feature
interactions and complex relationships without manual feature engineering.

## Final Model Training and Prediction

In [24]:
final_model = Pipeline(steps=[
    ("preprocessor", preprocessor_tree),
    ("regressor", HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=6,
        max_iter=300,
        random_state=42
    ))
])

In [25]:
final_model.fit(X, np.log1p(y))

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformer

In [26]:
test_ids = test_df["Id"]
X_test = test_df.drop(columns=["Id"])

X_test.shape, X.shape

((1459, 79), (1460, 79))

In [27]:
test_pred_log = final_model.predict(X_test)
test_pred = np.expm1(test_pred_log)

test_pred[:5]

array([119810.64913334, 153049.6177965 , 187503.15341268, 188009.71039395,
       180724.96508647])

In [30]:
import os

os.makedirs("data/predictions", exist_ok=True)
print(os.path.exists("data/predictions"))

True


In [31]:
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": test_pred
})

submission.to_csv("data/predictions/final_predictions.csv", index=False)

In [32]:
submission.head()
submission.shape

(1459, 2)